In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset,Dataset
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve
import os
import cv2
import numpy as np

from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from __main__ import SimplePreprocessor
# from __main__ import SimpleDatasetLoader



In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)
            data.append(image)
            labels.append(label)
            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))
        return (np.array(data), np.array(labels))
    



In [6]:

class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)


image_paths = list(paths.list_images("animals"))

# Initialize SimplePreprocessor and SimpleDatasetLoader and load data and labels
print('[INFO]: Images loading....')
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(image_paths, verbose=500)

# Reshape from (3000, 32, 32, 3) to (3000, 32*32*3=3072)
data = data.reshape((data.shape[0], 3072))

# Print information about memory consumption
# print('[INFO]: Features Matrix: {:.1f}MB'.format(float(data.nbytes / 1024*1000.0)))

# Encode labels as integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split data into training (75%) and testing (25%) data
X_train, X_test, y_train, y_test= train_test_split(data, labels, test_size=0.25, random_state=42)

print(X_train.shape)


[INFO]: Images loading....
[INFO]: Processed 500/3000
[INFO]: Processed 1000/3000
[INFO]: Processed 1500/3000
[INFO]: Processed 2000/3000
[INFO]: Processed 2500/3000
[INFO]: Processed 3000/3000
(2250, 3072)


In [7]:

class class_dataset(Dataset):
    def __init__(self,x_tensor,y_tensor):
        super().__init__()
        self.X = x_tensor
        self.Y = y_tensor

    def __getitem__(self,i):
        return (self.X[i],self.Y[i])
    def __len__(self):
        return len(self.X)

class SimpleClassificationNet(torch.nn.Module):
            
    def __init__(self):
        super().__init__() 
        self.linearLayer1 = nn.Linear(3072,1536)
        self.s1 = nn.ReLU()
        self.linearLayer2 = nn.Linear(1536,200)
        self.s2 = nn.ReLU()
        self.linearLayer3 = nn.Linear(200,3)
        self.s3 = nn.ReLU()



    def forward(self,x):
        first = self.linearLayer1(x)
        v = self.s1(first)
        second = self.linearLayer2(v)
        x = self.s2(second)
        y_hat = self.linearLayer3(x)
        return y_hat



In [ ]:

#Preparing PyTorch DataSets and DataLoaders

# Builds tensors from numpy arrays

x_train_t = torch.as_tensor(X_train).float()
y_train_t = torch.as_tensor(y_train).long()
x_val_t = torch.as_tensor(X_test).float()
y_val_t = torch.as_tensor(y_test).long()

# Builds dataset containing ALL data points

train_dataset = class_dataset(x_train_t,y_train_t)
val_dataset = class_dataset(x_val_t,y_val_t)

# Builds a loader of each set
train_load = DataLoader(dataset= train_dataset, batch_size = 10, shuffle = True)
val_load = DataLoader(dataset = val_dataset,batch_size = 10)

#model, optimizer and loss
model = SimpleClassificationNet().to(device)
stateDict=model.state_dict()


lr = 0.001
#  define optimizor here
optimizer = optim.SGD(model.parameters(),lr=lr)

# define loss here
loss_function = nn.CrossEntropyLoss()

#batch wise training loop
epochs = 100
train_losses = []
val_losses = []
best_accuracy=0
for epoch in range(epochs):  #epochs loop

    all_Y_train_epoch=np.array([]).reshape(0,1)
    all_Yhat_train_epoch=np.array([]).reshape(0,1)
    all_train_losses_epoch=np.array([])

    for X_train, Y_train in train_load:        #batch wise  training on train set
        model.train()
        X_train = X_train.to(device)
        Y_train = Y_train.to(device)
        logits = model(X_train)

        loss = loss_function(logits,Y_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    
#     backward passs
        
        #store metrics for all batches of current epoch 
        y_hat=F.softmax(logits,dim=-1)
        y_hat=y_hat.detach().cpu().numpy()
        y_hat=np.argmax(y_hat,axis=1)
        y_hat=y_hat.reshape(-1,1)

        Y_train=Y_train.detach().cpu().numpy()
        Y_train=Y_train.reshape(-1,1)
        all_Y_train_epoch=np.vstack((all_Y_train_epoch,Y_train))
        all_Yhat_train_epoch=np.vstack((all_Yhat_train_epoch,y_hat))   
        all_train_losses_epoch=np.append(all_train_losses_epoch,loss.item())     
        
    
    
    #computing metrics for current epoch
    train_losses.append(all_train_losses_epoch.mean()) #mean loss for all batches    
    acTrain=accuracy_score(all_Y_train_epoch, all_Yhat_train_epoch)
    cmTrain=confusion_matrix(all_Y_train_epoch, all_Yhat_train_epoch)
    print(cmTrain)

    #validation loop also bacth wise
    all_Y_val_epoch=np.array([]).reshape(0,1)
    all_Yhat_val_epoch=np.array([]).reshape(0,1)
    all_val_losses_epoch=np.array([])
    for X_val, Y_val in val_load:  #batch wise validation set predictions only
        model.eval()
        
        X_val = X_val.to(device)
        Y_val = Y_val.to(device)
        
        with torch.no_grad():            
            logits = model(X_val)           
            loss = loss_function(logits, Y_val)
        
        #store metrics for all batches of current epoch 
        y_hat_val=F.softmax(logits,dim=-1)
        y_hat_val=y_hat_val.detach().cpu().numpy()
        y_hat_val=np.argmax(y_hat_val,axis=1)
        y_hat_val=y_hat_val.reshape(-1,1)
        Y_val=Y_val.detach().cpu().numpy()
        Y_val=Y_val.reshape(-1,1)
        all_Y_val_epoch=np.vstack((all_Y_val_epoch,Y_val))
        all_Yhat_val_epoch=np.vstack((all_Yhat_val_epoch,y_hat_val))   
        all_val_losses_epoch=np.append(all_val_losses_epoch,loss.item())     
            

    #computing metrics for current epoch
    val_losses.append(all_val_losses_epoch.mean()) #mean loss for all batches    
    acVal=accuracy_score(all_Y_val_epoch, all_Yhat_val_epoch)
    cmVal=confusion_matrix(all_Y_val_epoch, all_Yhat_val_epoch)
    
    print(f"epoch= {epoch}, accuracyTrain= {acTrain}, accuracyVal= {acVal}, train_loss= {train_losses[epoch]}, validation_loss= {val_losses[epoch]}")
    
    #checkpointing training
    if(acVal>best_accuracy):
        checkpoint = {'epoch': epoch,'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),'loss': train_losses,
                      'val_loss': val_losses}
        torch.save(checkpoint,'best.pth')



#loading best model
checkpoint = torch.load('best.pth')
# Restore state for model and optimizer
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
total_epochs = checkpoint['epoch']
losses = checkpoint['loss']
val_losses = checkpoint['val_loss']

[[315 290 133]
 [286 316 149]
 [138 154 469]]
epoch= 0, accuracyTrain= 0.4888888888888889, accuracyVal= 0.5493333333333333, train_loss= 5.239010468986299, validation_loss= 0.8658189725875854
[[382 273  83]
 [299 335 117]
 [ 76 137 548]]
epoch= 1, accuracyTrain= 0.5622222222222222, accuracyVal= 0.5573333333333333, train_loss= 0.8543066169155968, validation_loss= 0.8635388024648031
[[392 279  67]
 [270 390  91]
 [ 76 105 580]]
epoch= 2, accuracyTrain= 0.6053333333333333, accuracyVal= 0.6093333333333333, train_loss= 0.805726036230723, validation_loss= 0.7744436275959015
[[409 261  68]
 [266 391  94]
 [ 65  98 598]]
epoch= 3, accuracyTrain= 0.6213333333333333, accuracyVal= 0.5746666666666667, train_loss= 0.7572512177626292, validation_loss= 0.8353607126077016
[[431 244  63]
 [253 400  98]
 [ 52  92 617]]
epoch= 4, accuracyTrain= 0.6435555555555555, accuracyVal= 0.592, train_loss= 0.7275443626774682, validation_loss= 0.802491182088852
[[443 246  49]
 [265 402  84]
 [ 47  84 630]]
epoch= 5, 

[[659  70   9]
 [ 77 660  14]
 [  4  22 735]]
epoch= 44, accuracyTrain= 0.9128888888888889, accuracyVal= 0.592, train_loss= 0.24295649013171594, validation_loss= 1.4919495075941085
